In [176]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.linear_model import PassiveAggressiveRegressor
from random import randint

In [177]:
env = gym.make('LunarLanderContinuous-v2')
env.seed(0)
state = env.reset()

##### state: [rover x, rover y, velocity_x, velocity_y, angle, leg 0 on ground, leg 1 on ground]
##### action: [main engine, left right engine]

In [178]:
observation, reward, done, info = env.step(action)

device(type='cuda')

In [169]:
class Agent:
    
    def __init__(self,model,actions:np.ndarray,turn_to_start_exploit):
        self.model = model
        self.actions = actions
        self.turn_to_start_exploit =turn_to_start_exploit
        
    def get_next_step(self,state,turn):
        actions_state = np.concatenate((self.actions, np.tile(state, (self.actions.shape[0],1))), axis=1)
        expected_scores = self.model.predict(actions_state)
        best = np.argmax(expected_scores) if turn>self.turn_to_start_exploit else randint(0,actions_state.shape[0]-1)
        score = expected_scores[best]
        best = np.argmax(expected_scores)
        action = self.actions[best]
        sample_for_training = actions_state[best]
        return action,sample_for_training,score

In [172]:
def train(gamma:float,stop_explor_stage:50):
    action_space = np.array([(engine_strangth/100,direction/100) for engine_strangth in range(0,101,10) for direction in range(-10,11,1)])
    model = PassiveAggressiveRegressor(warm_start=True,random_state=0)
    model.fit([[1 for i in range(34)]],[1])
    agent = Agent(model,action_space,stop_explor_stage)
    
    env = gym.make('LunarLanderContinuous-v2')
    env.seed(0)
    
          
    for i in range(3_000):
        if i==stop_explor_stage: print('stop exploring start explouting')
        over = False
        state = env.reset()
        states,actions,rewards,errors = [],[],[],[]
        while not over:
            
            action,for_training,predicted_score = agent.get_next_step(np.concatenate([state,state**2,state**3,state**4]),i)
            state, reward, over, _ = env.step(action)
            
            states.append(state)
            actions.append(for_training)
            rewards.append(reward)
            errors.append(abs(predicted_score-reward))
            agent.model.partial_fit(for_training.reshape(1, -1),[reward])
        
        Bellman_rewards = []
        rwords_for_long_train = np.array([])
        for reword in rewards[::-1]:
            rwords_for_long_train = np.append(rwords_for_long_train,np.array([reword]))
            Bellman_rewards.append(np.sum(rwords_for_long_train))
            rwords_for_long_train *= gamma
        Bellman_rewards = Bellman_rewards[::-1]
        agent.model.partial_fit(actions,Bellman_rewards)
        
        if np.max(rewards)>100:
            best_reward_condition = np.argmax(rewards)
            #print(actions[best_reward_condition-1:best_reward_condition+1])
        
        if i%10==0:
            print(f'last reward={reward}, mean reg error = {round(np.mean(errors))}, steps = {len(rewards)},total_score = {round(np.sum(rewards))},best score = {round(np.max(rewards))}')
            

In [175]:
train(gamma=0.9,stop_explor_stage=1000)

last reward=-100, mean reg error = 2, steps = 76,total_score = -119,best score = 23
last reward=-100, mean reg error = 5, steps = 71,total_score = -163,best score = 7
last reward=-100, mean reg error = 4, steps = 65,total_score = -117,best score = 23
last reward=-100, mean reg error = 8, steps = 64,total_score = -45,best score = 103
last reward=-100, mean reg error = 3, steps = 73,total_score = -182,best score = 6
last reward=-100, mean reg error = 4, steps = 91,total_score = -199,best score = 6
last reward=-100, mean reg error = 3, steps = 87,total_score = -138,best score = 7
last reward=-100, mean reg error = 4, steps = 49,total_score = -136,best score = 7
last reward=-100, mean reg error = 4, steps = 67,total_score = -138,best score = 8
last reward=-100, mean reg error = 4, steps = 58,total_score = -111,best score = 22
last reward=-100, mean reg error = 4, steps = 83,total_score = -165,best score = 6
last reward=-100, mean reg error = 3, steps = 81,total_score = -106,best score = 17